<a href="https://colab.research.google.com/github/VyacheslavGusev/Big_Data/blob/main/Big_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df_movies = pd.read_csv('u.item.csv', delimiter='|', encoding = "ISO-8859-1", header = None, names = ['movie id', 'movie title', 'release date', 'video release date',
                                                                                       'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation',
                                                                                       'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                                                                                       'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                                                                                       'Thriller','War', 'Western'] )

In [ ]:
df_rating = pd.read_csv('u.data.csv', delimiter='\t', names = ['user id','item id', 'rating', 'timestamp'])

In [ ]:
df_rating.head()

,user id,item id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
df_movies.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Находим самого активного пользователя

In [ ]:
df_rating.groupby('user id')[['rating']].agg('count').sort_values('rating', ascending=False).head(1)


,rating
user id,
405,737


Оставляем фильмы, которые оценил пользователь

In [ ]:
df_rating405 = df_rating[df_rating['user id'] ==405]

In [ ]:
df_rating405.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 737 entries, 12276 to 99756
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   user id    737 non-null    int64
 1   item id    737 non-null    int64
 2   rating     737 non-null    int64
 3   timestamp  737 non-null    int64
dtypes: int64(4)
memory usage: 28.8 KB


In [ ]:
df_rating405.head()

,user id,item id,rating,timestamp
12276,405,56,4,885544911
12383,405,592,1,885548670
12430,405,1582,1,885548670
12449,405,171,1,885549544
12460,405,580,1,885547447


In [ ]:
df_merged = df_rating405.merge(df_movies[['movie id','unknown', 'Action', 'Adventure', 'Animation',
                              'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                              'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                              'Thriller','War', 'Western']], left_on = 'item id', right_on = 'movie id', how = 'left')

In [ ]:
df_merged = df_merged.merge(df_rating.groupby('item id')[['rating']].agg('count'), left_on = 'item id', right_on = 'item id', how = 'left')

In [ ]:
df_merged = df_merged.merge(df_rating.groupby('item id')[['rating']].agg('sum'), left_on = 'item id', right_on = 'item id', how = 'left')


In [ ]:
df_merged.head()

,user id,item id,rating_x,timestamp,movie id,unknown,Action,Adventure,Animation,Childrens,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating_y,rating
0,405,56,4,885544911,56,0,0,0,0,0,...,0,0,0,0,0,0,0,0,394,1600
1,405,592,1,885548670,592,0,0,0,0,0,...,0,0,1,0,0,1,0,0,9,30
2,405,1582,1,885548670,1582,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3,405,171,1,885549544,171,0,0,0,0,0,...,0,0,0,0,1,0,0,0,65,252
4,405,580,1,885547447,580,0,0,0,0,0,...,0,0,0,1,0,0,0,0,32,108


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
df_merged.columns

Index(['user id', 'item id', 'rating_x', 'timestamp', 'movie id', 'unknown',
       'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western',
       'rating_y', 'rating'],
      dtype='object')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_merged[['unknown',
       'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western',
       'rating_y', 'rating']], df_merged['rating_x'], test_size=0.2)

Приводим данные к одному размеру

In [ ]:
sc = StandardScaler()

In [ ]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(X_train, y_train)


LogisticRegression()

In [ ]:
lr.score(X_test, y_test)


0.6486486486486487

Загружаем Spark

In [ ]:
!apt-get update

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,360 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,299 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 4,999 kB i

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://downloads.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz

In [ ]:
!tar -xvf spark-3.4.0-bin-hadoop3.tgz

spark-3.4.0-bin-hadoop3/
spark-3.4.0-bin-hadoop3/data/
spark-3.4.0-bin-hadoop3/data/streaming/
spark-3.4.0-bin-hadoop3/data/streaming/AFINN-111.txt
spark-3.4.0-bin-hadoop3/data/graphx/
spark-3.4.0-bin-hadoop3/data/graphx/followers.txt
spark-3.4.0-bin-hadoop3/data/graphx/users.txt
spark-3.4.0-bin-hadoop3/data/mllib/
spark-3.4.0-bin-hadoop3/data/mllib/als/
spark-3.4.0-bin-hadoop3/data/mllib/als/test.data
spark-3.4.0-bin-hadoop3/data/mllib/als/sample_movielens_ratings.txt
spark-3.4.0-bin-hadoop3/data/mllib/gmm_data.txt
spark-3.4.0-bin-hadoop3/data/mllib/sample_libsvm_data.txt
spark-3.4.0-bin-hadoop3/data/mllib/sample_svm_data.txt
spark-3.4.0-bin-hadoop3/data/mllib/sample_fpgrowth.txt
spark-3.4.0-bin-hadoop3/data/mllib/kmeans_data.txt
spark-3.4.0-bin-hadoop3/data/mllib/pic_data.txt
spark-3.4.0-bin-hadoop3/data/mllib/streaming_kmeans_data_test.txt
spark-3.4.0-bin-hadoop3/data/mllib/sample_linear_regression_data.txt
spark-3.4.0-bin-hadoop3/data/mllib/ridge-data/
spark-3.4.0-bin-hadoop3/data/

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [ ]:
movies_df = spark.read.option("delimiter", '|').csv('u.item.csv', inferSchema= True, header= False).toDF('movie_id', 'movie title', 'release date', 'video release date',
                                                                                               'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation',
                                                                                               'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                                                                                               'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                                                                                               'Thriller','War', 'Western')

In [ ]:
movies_df.show()

+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_id|         movie title|release date|video release date|            IMDb URL|unknown|Action|Adventure|Animation|Childrens|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|       1|    Toy Story (1995)| 01-Jan-1995|              null|http://us.imdb.co...|      0|     0|        0|        1|        1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|       2|    GoldenEye (1995)| 01-Jan-1995|    

In [ ]:
rating_df = spark.read.option("delimiter", '\t').csv('u.data.csv', inferSchema= True, header= False).toDF('user_id','item_id', 'rating', 'timestamp')

In [ ]:
rating_df.show()

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
|    298|    474|     4|884182806|
|    115|    265|     2|881171488|
|    253|    465|     5|891628467|
|    305|    451|     3|886324817|
|      6|     86|     3|883603013|
|     62|    257|     2|879372434|
|    286|   1014|     5|879781125|
|    200|    222|     5|876042340|
|    210|     40|     3|891035994|
|    224|     29|     3|888104457|
|    303|    785|     3|879485318|
|    122|    387|     5|879270459|
|    194|    274|     2|879539794|
|    291|   1042|     4|874834944|
|    234|   1184|     2|892079237|
+-------+-------+------+---------+
only showing top 20 rows



In [ ]:
merged_df = movies_df.join(rating_df, movies_df.movie_id == rating_df.item_id, how ='left')

In [ ]:
merged_df.show()

+--------+----------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+-------+-------+------+---------+
|movie_id|     movie title|release date|video release date|            IMDb URL|unknown|Action|Adventure|Animation|Childrens|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|user_id|item_id|rating|timestamp|
+--------+----------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+-------+-------+------+---------+
|       1|Toy Story (1995)| 01-Jan-1995|              null|http://us.imdb.co...|      0|     0|        0|        1|        1|     1|    0|          0|    0|      0|        0|     0|      0|      0|  

Средствами спарка вывести среднюю оценку для каждого фильма

In [ ]:
merged_df.groupby('movie_id', 'movie title').avg('rating').show()

+--------+--------------------+------------------+
|movie_id|         movie title|       avg(rating)|
+--------+--------------------+------------------+
|     170|Cinema Paradiso (...|4.1735537190082646|
|     171| Delicatessen (1991)| 3.876923076923077|
|     279|Once Upon a Time....|3.2857142857142856|
|     394|Radioland Murders...|3.3333333333333335|
|     660|Fried Green Tomat...| 3.764705882352941|
|     708|Sex, Lies, and Vi...|3.4752475247524752|
|     873|Picture Perfect (...| 2.962962962962963|
|    1512|World of Apu, The...|               4.0|
|     517|    Manhattan (1979)| 3.769230769230769|
|     586|Terminal Velocity...| 2.676470588235294|
|     648|Quiet Man, The (1...| 4.029850746268656|
|    1437|House Party 3 (1994)|               2.0|
|    1618|King of New York ...|               1.0|
|      43|   Disclosure (1994)|               3.0|
|     923|Raise the Red Lan...| 4.155172413793103|
|     961|      Orlando (1993)|3.6176470588235294|
|     981|Dangerous Ground ...|

Посчитайте средствами спарка среднюю оценку для каждого жанра

In [ ]:
merged_df.columns

['movie_id',
 'movie title',
 'release date',
 'video release date',
 'IMDb URL',
 'unknown',
 'Action',
 'Adventure',
 'Animation',
 'Childrens',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western',
 'user_id',
 'item_id',
 'rating',
 'timestamp']

In [ ]:
merged_df.where((merged_df['unknown'] == 1)).agg({'rating': 'avg'}).show()

+-----------+
|avg(rating)|
+-----------+
|        3.2|
+-----------+



In [ ]:
merged_df.where((merged_df['Action'] == 1)).agg({'rating': 'avg'}).show()

+-----------------+
|      avg(rating)|
+-----------------+
|3.480245417953027|
+-----------------+



In [ ]:
merged_df.where((merged_df['Adventure'] == 1)).agg({'rating': 'avg'}).show()

+-----------------+
|      avg(rating)|
+-----------------+
|3.503526503308369|
+-----------------+



In [ ]:
merged_df.where((merged_df['Animation'] == 1)).agg({'rating': 'avg'}).show()

+------------------+
|       avg(rating)|
+------------------+
|3.5766990291262135|
+------------------+



In [ ]:
merged_df.where((merged_df['Childrens'] == 1)).agg({'rating': 'avg'}).show()

+------------------+
|       avg(rating)|
+------------------+
|3.3532442216652742|
+------------------+



In [ ]:
merged_df.where((merged_df['Comedy'] == 1)).agg({'rating': 'avg'}).show()

+------------------+
|       avg(rating)|
+------------------+
|3.3940734781442745|
+------------------+



In [ ]:
merged_df.where((merged_df['Crime'] == 1)).agg({'rating': 'avg'}).show()

+------------------+
|       avg(rating)|
+------------------+
|3.6322780881440098|
+------------------+



In [ ]:
merged_df.where((merged_df['Documentary'] == 1)).agg({'rating': 'avg'}).show()

+------------------+
|       avg(rating)|
+------------------+
|3.6728232189973613|
+------------------+



In [ ]:
merged_df.where((merged_df['Drama'] == 1)).agg({'rating': 'avg'}).show()

+------------------+
|       avg(rating)|
+------------------+
|3.6873793708484772|
+------------------+



In [ ]:
merged_df.where((merged_df['Film-Noir'] == 1)).agg({'rating': 'avg'}).show()

+------------------+
|       avg(rating)|
+------------------+
|3.9215233698788228|
+------------------+



In [ ]:
merged_df.where((merged_df['Horror'] == 1)).agg({'rating': 'avg'}).show()

+------------------+
|       avg(rating)|
+------------------+
|3.2903893172841827|
+------------------+



In [ ]:
merged_df.where((merged_df['Musical'] == 1)).agg({'rating': 'avg'}).show()

+-----------------+
|      avg(rating)|
+-----------------+
|3.521396851029471|
+-----------------+



In [ ]:
merged_df.where((merged_df['Mystery'] == 1)).agg({'rating': 'avg'}).show()

+----------------+
|     avg(rating)|
+----------------+
|3.63813155386082|
+----------------+



In [ ]:
merged_df.where((merged_df['Romance'] == 1)).agg({'rating': 'avg'}).show()

+-----------------+
|      avg(rating)|
+-----------------+
|3.621704948358255|
+-----------------+



In [ ]:
merged_df.where((merged_df['Sci-Fi'] == 1)).agg({'rating': 'avg'}).show()

+------------------+
|       avg(rating)|
+------------------+
|3.5607227022780834|
+------------------+



In [ ]:
merged_df.where((merged_df['Thriller'] == 1)).agg({'rating': 'avg'}).show()

+------------------+
|       avg(rating)|
+------------------+
|3.5090069495245064|
+------------------+



In [ ]:
merged_df.where((merged_df['War'] == 1)).agg({'rating': 'avg'}).show()

+-----------------+
|      avg(rating)|
+-----------------+
|3.815811874866993|
+-----------------+



In [ ]:
merged_df.where((merged_df['Western'] == 1)).agg({'rating': 'avg'}).show()

+------------------+
|       avg(rating)|
+------------------+
|3.6132686084142396|
+------------------+



В спарке получить 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами (по количеству оценок)

In [ ]:
rating_count = merged_df.groupBy('movie title').count()

In [ ]:
rating_count.orderBy('count', ascending = False).show(5)

+--------------------+-----+
|         movie title|count|
+--------------------+-----+
|    Star Wars (1977)|  583|
|      Contact (1997)|  509|
|        Fargo (1996)|  508|
|Return of the Jed...|  507|
|    Liar Liar (1997)|  485|
+--------------------+-----+
only showing top 5 rows



In [ ]:
rating_count.orderBy('count', ascending = True).show(5)

+--------------------+-----+
|         movie title|count|
+--------------------+-----+
|    Fear, The (1995)|    1|
|Aiqing wansui (1994)|    1|
|Next Step, The (1...|    1|
|Vie est belle, La...|    1|
| Mad Dog Time (1996)|    1|
+--------------------+-----+
only showing top 5 rows

